This is my working notebook where I will explore and create models. The most relavant information from here will be transfered to final_report.ipynb where I can more concisely explain my finding and give actionable data.

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import prepare
import acquire
import random
from matplotlib import style
style.use('ggplot')
import itertools

ModuleNotFoundError: No module named 'prepare'

Acquire/Prep. 
    It is worth noting that prepare.py relies on acquire.py to pull in the data which it preps

In [1]:
# this function call acquires and preps in one motion, worth noting that 
# get_titanic_data() is called within prep_titanic() and will save a .csv of the 
# raw data
titanic = prepare.prep_titanic()
#look at the data
titanic.head()

NameError: name 'prepare' is not defined